In [1]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
sys.path.append('/Users/kristine/Library/Python/3.8/lib/python/site-packages')
import ee
import numpy as np
import pandas as pd
import time

In [2]:
#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [5]:
#Load image collection
annual_temperature = ee.ImageCollection('projects/resource-watch-gee/Facebook/TemperatureAnalysis/GHCN_CAMS_annual')

#Save projection
projection = annual_temperature.first().projection().getInfo()
projection_gee = annual_temperature.first().projection()
crs = projection.get('crs')
crsTransform = projection.get('transform')
scale = annual_temperature.first().projection().nominalScale().getInfo()
print('CRS and Transform: ',crs, crsTransform)

#Convert imageCollection to Image, this will make it easier to use reduceRegions to calculate the average
#temperature for each region
band_names = annual_temperature.aggregate_array('system:index').map(lambda x: ee.String(x).slice(-4))
annual_temperature = annual_temperature.toBands()
annual_temperature = annual_temperature.rename(band_names)
annual_temperature = annual_temperature.add(ee.Image.constant(-273.15))

#Define function to merge shapes that may be split up due to large geometries and calculate average
#temperature anomaly
def calculate_average_temperature_per_feature(feature_collection, output_name, output_folder):
    #Calculate average temperature anomaly for each year for each country
    average_annual_temperature = annual_temperature.reduceRegions(feature_collection, 
                                                          ee.Reducer.mean(), 
                                                          crs=crs, crsTransform=crsTransform, 
                                                          tileScale=8)
    #Drop geometry information
    average_annual_temperature = average_annual_temperature.map(lambda x: x.select(x.propertyNames(),
                                                                           retainGeometry=False))

    #Export to Google Drive
    export_results_task = ee.batch.Export.table.toDrive(
        collection = average_annual_temperature, 
        description = output_name, 
        fileNamePrefix = output_name,
        folder = output_folder)

    export_results_task.start()

    #Wait for export to finish
    while export_results_task.active():
        print('Polling for task (id: {}).'.format(export_results_task.id))
        time.sleep(30)
    print('Done with export.')

    
#Define countries to calculate statistics for
country_list = ["USA","GBR","FRA","DEU","CAN", "SWE", "BRA", "MEX", "BEL", "IRL", "NLD", "NGA", 
                "SAU", "ZAF", "ESP", "IND", "IDN", "TWN"]
country_list = ee.List(country_list)


#Load country data, filter to desired ISO Codes
countries = ee.FeatureCollection("projects/resource-watch-gee/gadm36_0_simplified")
countries = ee.FeatureCollection(country_list.map(lambda x: countries.filterMetadata('GID_0','equals',
                                                                                     ee.String(x)))).flatten()
#Use function to calculate temperature anomalies and export
calculate_average_temperature_per_feature(countries, 
                                          output_name='Average_Temperature_Country_Level', 
                                          output_folder='Facebook')


#Load state data, filter to desired ISO Codes
states = ee.FeatureCollection("projects/resource-watch-gee/gadm36_1_simplified")
states = ee.FeatureCollection(country_list.map(lambda x: states.filterMetadata('GID_0','equals',
                                                                                  ee.String(x)))).flatten()
#Use function to calculate temperature anomalies and export
calculate_average_temperature_per_feature(states, 
                                          output_name='Average_Temperature_State_Level', 
                                          output_folder='Facebook')


    

CRS and Transform:  EPSG:4326 [0.5, 0, -180, 0, -0.5, 90]
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Polling for task (id: A6CQIZFRPUNLB6PBOSE27YYX).
Done with export.
Polling for task (id: 4VNZNBAQYDRPQJBOC42SZJ63).
Polling for task (id: 4VNZNBAQYDRPQJBOC42SZJ63).
Done with export.


In [ ]:
land = ee.FeatureCollection("projects/resource-watch-gee/gadm36_0_simplified")
land = land.filterMetadata('GID_0','not_equals','ATA')
global_geometry = land.geometry()

global_temperature = annual_temperature.reduceRegion(reducer=ee.Reducer.mean(), 
                                                     geometry=global_geometry, 
                                                     crs=crs, crsTransform=crsTransform, 
                                                     bestEffort=True, maxPixels=1e13, tileScale=4)
global_temperature_feature = ee.Feature(None,global_temperature)
global_temperature_feature_collection = ee.FeatureCollection(global_temperature_feature)

output_name='Average_Temperature_Global_Level'
output_folder='Facebook'

#Export to Google Drive
export_results_task = ee.batch.Export.table.toDrive(
    collection = global_temperature_feature_collection, 
    description = output_name, 
    fileNamePrefix = output_name,
    folder = output_folder)

export_results_task.start()

#Wait for export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')


Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id: AF7KLH7U7BAP3XCZJTFWOL46).
Polling for task (id